
# Week 1 Assignment, **Task 2**  

`pip install cobra pandas`

In [21]:
# Imports and version checks
import sys
from pathlib import Path

import pandas as pd
import cobra



## ⚙️ Configuration
Set paths to the model and the expression CSV.  
You can toggle saving artifacts (CSV and constrained model). The assignment only **requires printing** a simple table; saving is optional.


In [ ]:

# Paths
MODEL_PATH = Path("e_coli_core.json")
EXPR_PATH  = Path("e_coli_core_expression.csv")

# Special rules
GLUCOSE_EX_ID = "EX_glc__D_e"
HIGH_DEFAULT  = 1000.0

# If you want to save the output files
SAVE_FILES = False
OUT_BOUNDS_CSV = Path("task2_bounds_table.csv")
OUT_SUMMARY_CSV = Path("task2_change_summary.csv")
OUT_MODEL_JSON  = Path("e_coli_core_constrained.json")


## 🧰 Helper functions (small & testable)
Scientific notes are embedded as docstrings. Functions are intentionally short for clarity.


In [ ]:
from dataclasses import dataclass
from typing import Dict, Tuple, Optional, Iterable

import csv

# Data structures

@dataclass
class Bounds:
    lb: float
    ub: float

@dataclass
class BoundsChange:
    reaction_id: str
    reaction_name: str
    before: Bounds
    after: Bounds
    reason: str


def load_model(model_path: Path):
    """Load a COBRA JSON model.
    
    Parameters
    ----------
    model_path : Path
        Path to 'e_coli_core.json' (from the first practical).
    
    Returns
    -------
    cobra.Model
    """
    if not model_path.exists():
        raise FileNotFoundError(f"Model file not found: {model_path}")
    return cobra.io.load_json_model(str(model_path))

def save_model(model, out_path: Path) -> None:
    """Save a COBRA model to JSON (optional for the assignment)."""
    cobra.io.save_json_model(model, str(out_path))

def read_expression_csv(expr_path: Path) -> Tuple[Dict[str, float], Dict[str, float]]:
    """Read maximal activities from a CSV.
    
    Expected format (at least two columns):
        key, value, ...
    'key' can be either a reaction **ID** (e.g., 'PGK') or a reaction **NAME** (e.g., 'Phosphoglycerate kinase').
    'value' must be a non-negative float (mmol/gDW/h).
    
    Returns
    -------
    expr_by_id : Dict[str, float]
        Mapping reaction ID -> activity.
    expr_by_name : Dict[str, float]
        Mapping reaction NAME (lowercased) -> activity.
    """
    if not expr_path.exists():
        raise FileNotFoundError(f"Expression CSV not found: {expr_path}")
    
    expr_by_id: Dict[str, float] = {}
    expr_by_name: Dict[str, float] = {}
    
    with open(expr_path, mode="r", newline="") as f:
        reader = csv.reader(f)
        for row in reader:
            if not row or len(row) < 2:
                continue
            key = (row[0] or "").strip()
            try:
                val = float(row[1])
            except Exception:
                # skip header/malformed row
                continue
            if val < 0:
                continue
            if not key:
                continue
            # Heuristic: names often contain spaces or lowercase letters
            if (" " in key) or (key.lower() != key.upper()):
                expr_by_name[key.lower()] = val
            else:
                expr_by_id[key] = val
    return expr_by_id, expr_by_name


# ---------- Small scientific helpers ----------

def build_name_index(model) -> Dict[str, object]:
    """Map reaction NAME (lowercased) -> reaction object."""
    return { (rxn.name or '').lower(): rxn for rxn in model.reactions if rxn.name }

def get_bounds(rxn) -> Bounds:
    return Bounds(lb=float(rxn.lower_bound), ub=float(rxn.upper_bound))

def set_bounds(rxn, lb: float, ub: float) -> None:
    rxn.lower_bound = float(lb)
    rxn.upper_bound = float(ub)

def is_reversible_from_bounds(rxn) -> bool:
    """Infer reversibility from current bounds.
    
    Scientific rationale: In COBRA models, a reaction is considered reversible if its feasible
    flux interval straddles zero (lb < 0 < ub). If lb >= 0, we treat it as irreversible forward.
    """
    b = get_bounds(rxn)
    return (b.lb < 0.0) and (b.ub > 0.0)

def clamp_irreversible_lower_bound(rxn) -> None:
    """Ensure irreversible reactions have non-negative lower bound (>= 0)."""
    if rxn.lower_bound < 0:
        rxn.lower_bound = 0.0


# ---------- Rule applications (each is a tiny, testable method) ----------

def apply_rule_reversible(rxn, A: float) -> BoundsChange:
    """Reversible reaction: set bounds to [-A, +A]."""
    before = get_bounds(rxn)
    set_bounds(rxn, lb=-A, ub=+A)
    after = get_bounds(rxn)
    return BoundsChange(rxn.id, rxn.name or '', before, after, reason='expr_rule_reversible')

def apply_rule_irreversible(rxn, A: float) -> BoundsChange:
    """Irreversible reaction: set bounds to [0, A]."""
    before = get_bounds(rxn)
    clamp_irreversible_lower_bound(rxn)
    set_bounds(rxn, lb=max(0.0, rxn.lower_bound), ub=A)
    after = get_bounds(rxn)
    return BoundsChange(rxn.id, rxn.name or '', before, after, reason='expr_rule_irreversible')

def apply_rule_atpm_upper_only(rxn, A: float) -> BoundsChange:
    """ATPM special case: keep lower bound (maintenance) unchanged; set only upper bound to A."""
    before = get_bounds(rxn)
    set_bounds(rxn, lb=before.lb, ub=A)
    after = get_bounds(rxn)
    return BoundsChange(rxn.id, rxn.name or '', before, after, reason='expr_rule_ATPM_upper_only')

def apply_rule_glucose_exchange(rxn, high_default: float = 1000.0) -> BoundsChange:
    """Glucose exchange special case: remove tight cap; set ±high_default."""
    before = get_bounds(rxn)
    set_bounds(rxn, lb=-high_default, ub=+high_default)
    after = get_bounds(rxn)
    return BoundsChange(rxn.id, rxn.name or '', before, after, reason='glucose_exchange_rule')


# ---------- Orchestration to apply all constraints ----------

def apply_activity_constraints(model,
                               expr_by_id: Dict[str, float],
                               expr_by_name: Dict[str, float],
                               glucose_ex_id: str = 'EX_glc__D_e',
                               high_default: float = 1000.0) -> pd.DataFrame:
    """Apply Task 2 constraints in-place and return an audit DataFrame.
    
    Steps
    -----
    1) Determine initial reversibility per reaction from current bounds.
    2) Apply expression data by **reaction ID**.
    3) Apply expression data by **reaction NAME** (if not already updated by ID).
    4) Apply **glucose exchange** rule (±high_default).
    5) Reactions **without data** remain unchanged.
    
    Returns
    -------
    audit_df : pd.DataFrame
        Columns: ['reaction_id','reaction_name','before_lb','before_ub','after_lb','after_ub','reason']
    """
    name_index = build_name_index(model)
    initial_rev = {rxn.id: is_reversible_from_bounds(rxn) for rxn in model.reactions}
    changes = []
    updated_ids = set()
    
    # 2) By ID
    for rid, A in expr_by_id.items():
        rxn = model.reactions.get_by_id(rid) if rid in model.reactions else None
        if rxn is None:
            continue
        if rxn.id == 'ATPM':
            c = apply_rule_atpm_upper_only(rxn, A)
        else:
            c = apply_rule_reversible(rxn, A) if initial_rev.get(rxn.id, False) else apply_rule_irreversible(rxn, A)
        changes.append(c)
        updated_ids.add(rxn.id)
    
    # 3) By NAME
    for name_lower, A in expr_by_name.items():
        rxn = name_index.get(name_lower)
        if rxn is None or rxn.id in updated_ids:
            continue
        if rxn.id == 'ATPM':
            c = apply_rule_atpm_upper_only(rxn, A)
        else:
            c = apply_rule_reversible(rxn, A) if initial_rev.get(rxn.id, False) else apply_rule_irreversible(rxn, A)
        changes.append(c)
        updated_ids.add(rxn.id)
    
    # 4) Glucose exchange rule
    if glucose_ex_id in model.reactions:
        rxn = model.reactions.get_by_id(glucose_ex_id)
        changes.append(apply_rule_glucose_exchange(rxn, high_default=high_default))
    
    # Build audit DataFrame
    audit_df = pd.DataFrame([{
        "reaction_id": c.reaction_id,
        "reaction_name": c.reaction_name,
        "before_lb": c.before.lb,
        "before_ub": c.before.ub,
        "after_lb": c.after.lb,
        "after_ub": c.after.ub,
        "reason": c.reason
    } for c in changes])
    return audit_df


# ---------- Reporting ----------

def make_bounds_table(model) -> pd.DataFrame:
    """Return [reaction_id, reaction_name, lower_bound, upper_bound] for all reactions."""
    rows = []
    for rxn in sorted(model.reactions, key=lambda r: r.id):
        rows.append({
            "reaction_id": rxn.id,
            "reaction_name": rxn.name or "",
            "lower_bound": float(rxn.lower_bound),
            "upper_bound": float(rxn.upper_bound)
        })
    return pd.DataFrame(rows)

def print_bounds_table(df: pd.DataFrame) -> None:
    """Plain print (as required in the assignment)."""
    print("reaction_id\treaction_name\tlower_bound\tupper_bound")
    for _, row in df.sort_values("reaction_id").iterrows():
        print(f"{row['reaction_id']}\t{row['reaction_name']}\t{row['lower_bound']:.6g}\t{row['upper_bound']:.6g}")

def summarize_mapping_stats(audit_df: Optional[pd.DataFrame]) -> pd.DataFrame:
    """How many reactions were changed under each rule (reason)."""
    if audit_df is None or audit_df.empty:
        return pd.DataFrame({"reason": [], "n": []})
    return (audit_df["reason"].value_counts()
            .rename_axis("reason")
            .reset_index(name="n"))



## 🧪 Load the model


In [15]:
model = load_model(MODEL_PATH)
print(f"Model loaded with {len(model.reactions)} reactions and {len(model.metabolites)} metabolites.")


Model loaded with 95 reactions and 72 metabolites.



## 📄 Load the expression-derived maximal activities


In [16]:
expr_by_id, expr_by_name = read_expression_csv(EXPR_PATH)
print(f"Loaded {len(expr_by_id)} entries by ID and {len(expr_by_name)} entries by NAME from CSV.")
if len(expr_by_id) + len(expr_by_name) == 0:
    print("⚠️ No valid rows found. Check your CSV: first column key (ID or Name), second column numeric value.")


Loaded 0 entries by ID and 73 entries by NAME from CSV.



## 🔧 Apply the constraints (Task 2 rules)
- Reversible: **[-A, +A]**  
- Irreversible: **[0, A]**  
- `ATPM`: keep **lower bound**; set **upper bound = A** if provided  
- `EX_glc__D_e` (glucose exchange): set to **±1000** (remove tight cap)  
- No data: leave original bounds


In [17]:

audit_df = apply_activity_constraints(
    model,
    expr_by_id=expr_by_id,
    expr_by_name=expr_by_name,
    glucose_ex_id=GLUCOSE_EX_ID,
    high_default=HIGH_DEFAULT
)
print("Applied constraints.")
display(audit_df.head(10))

summary_df = summarize_mapping_stats(audit_df)
print("\nChange summary (by reason):")
display(summary_df)


Applied constraints.


,reaction_id,reaction_name,before_lb,before_ub,after_lb,after_ub,reason
0,EX_glc__D_e,D-Glucose exchange,-10.0,1000.0,-1000.0,1000.0,glucose_exchange_rule



Change summary (by reason):


,reason,n
0,glucose_exchange_rule,1



## 📋 Bounds table (assignment requirement)
The assignment requests a **simple printout** of each reaction’s lower and upper flux bounds.  
We also display the first rows as a DataFrame for convenience.


In [18]:

bounds_df = make_bounds_table(model)

# Simple printout (required)
print_bounds_table(bounds_df)

# Friendly preview
print("\nPreview (first 20 rows):")
display(bounds_df.head(20))


reaction_id	reaction_name	lower_bound	upper_bound
ACALD	Acetaldehyde dehydrogenase (acetylating)	-1000	1000
ACALDt	Acetaldehyde reversible transport	-1000	1000
ACKr	Acetate kinase	-1000	1000
ACONTa	Aconitase (half-reaction A, Citrate hydro-lyase)	-1000	1000
ACONTb	Aconitase (half-reaction B, Isocitrate hydro-lyase)	-1000	1000
ACt2r	Acetate reversible transport via proton symport	-1000	1000
ADK1	Adenylate kinase	-1000	1000
AKGDH	2-Oxogluterate dehydrogenase	0	1000
AKGt2r	2 oxoglutarate reversible transport via symport	-1000	1000
ALCD2x	Alcohol dehydrogenase (ethanol)	-1000	1000
ATPM	ATP maintenance requirement	8.39	1000
ATPS4r	ATP synthase (four protons for one ATP)	-1000	1000
BIOMASS_Ecoli_core_w_GAM	Biomass Objective Function with GAM	0	1000
CO2t	CO2 transporter via diffusion	-1000	1000
CS	Citrate synthase	0	1000
CYTBD	Cytochrome oxidase bd (ubiquinol-8: 2 protons)	0	1000
D_LACt2	D lactate transport via proton symport	-1000	1000
ENO	Enolase	-1000	1000
ETOHt2r	Ethanol reversible transp

,reaction_id,reaction_name,lower_bound,upper_bound
0,ACALD,Acetaldehyde dehydrogenase (acetylating),-1000.00,1000.0
1,ACALDt,Acetaldehyde reversible transport,-1000.00,1000.0
2,ACKr,Acetate kinase,-1000.00,1000.0
3,ACONTa,"Aconitase (half-reaction A, Citrate hydro-lyase)",-1000.00,1000.0
4,ACONTb,"Aconitase (half-reaction B, Isocitrate hydro-l...",-1000.00,1000.0
5,ACt2r,Acetate reversible transport via proton symport,-1000.00,1000.0
6,ADK1,Adenylate kinase,-1000.00,1000.0
7,AKGDH,2-Oxogluterate dehydrogenase,0.00,1000.0
8,AKGt2r,2 oxoglutarate reversible transport via symport,-1000.00,1000.0
9,ALCD2x,Alcohol dehydrogenase (ethanol),-1000.00,1000.0



## 💾 Optional: Save artifacts (set `SAVE_FILES = True` above)
- `task2_bounds_table.csv`
- `e_coli_core_constrained.json`
- `task2_change_summary.csv`


In [19]:

if SAVE_FILES:
    bounds_df.to_csv(OUT_BOUNDS_CSV, index=False)
    summary_df.to_csv(OUT_SUMMARY_CSV, index=False)
    save_model(model, OUT_MODEL_JSON)
    print(f"Saved bounds table   -> {OUT_BOUNDS_CSV}")
    print(f"Saved change summary -> {OUT_SUMMARY_CSV}")
    print(f"Saved constrained model -> {OUT_MODEL_JSON}")
else:
    print("Saving skipped (SAVE_FILES=False).")


Saving skipped (SAVE_FILES=False).



## ✅ Quick sanity checks
These simple checks help you confirm the special rules were applied.


In [20]:

def show_bounds(rid: str):
    if rid not in model.reactions:
        print(f"{rid} not found in model")
        return
    rxn = model.reactions.get_by_id(rid)
    print(rid, rxn.name, float(rxn.lower_bound), float(rxn.upper_bound))

# Check glucose exchange
show_bounds(GLUCOSE_EX_ID)

# Check ATPM (lower bound should be preserved)
show_bounds("ATPM")


EX_glc__D_e D-Glucose exchange -1000.0 1000.0
ATPM ATP maintenance requirement 8.39 1000.0



## ▶️ Next steps (for Tasks 3–4)
- Use `e_coli_core_constrained.json` to run **FVA** and **FBA**.
- For FVA (Task 3): `cobra.flux_analysis.variability.flux_variability_analysis(...)`
- For FBA (Task 4): `solution = model.optimize()` then inspect `solution.objective_value` and fluxes.
